## Dealing with categorical variable

## 1. Frequent category imputation

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('house_prediction.csv')

In [3]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
columns_of_interest = ['BsmtQual','FireplaceQu','GarageType','SalePrice']

In [5]:
df = df[columns_of_interest]

In [6]:
df

,BsmtQual,FireplaceQu,GarageType,SalePrice
0,Gd,NaN,Attchd,208500
1,Gd,TA,Attchd,181500
2,Gd,TA,Attchd,223500
3,TA,Gd,Detchd,140000
4,Gd,TA,Attchd,250000
...,...,...,...,...
1455,Gd,TA,Attchd,175000
1456,Gd,TA,Attchd,210000
1457,TA,Gd,Attchd,266500
1458,TA,NaN,Attchd,142125


In [7]:
## Proportion of missing values
## BsmtQual and GarageType are the categorical variable with less number of missing values. Hence , we can replace them with the most frequent value
df.isnull().sum()/df.shape[0]

BsmtQual       0.025342
FireplaceQu    0.472603
GarageType     0.055479
SalePrice      0.000000
dtype: float64

In [8]:
bsmtqual_mostfrequent = df.BsmtQual.value_counts().index[0]
garagetype_mostfrequent = df.GarageType.value_counts().index[0]
bsmtqual_mostfrequent, garagetype_mostfrequent

('TA', 'Attchd')

In [9]:
df = df.copy()
df['BsmtQual'] = df.BsmtQual.fillna(bsmtqual_mostfrequent)
df['GarageType'] = df.BsmtQual.fillna(garagetype_mostfrequent)

In [10]:
df

,BsmtQual,FireplaceQu,GarageType,SalePrice
0,Gd,NaN,Gd,208500
1,Gd,TA,Gd,181500
2,Gd,TA,Gd,223500
3,TA,Gd,TA,140000
4,Gd,TA,Gd,250000
...,...,...,...,...
1455,Gd,TA,Gd,175000
1456,Gd,TA,Gd,210000
1457,TA,Gd,TA,266500
1458,TA,NaN,TA,142125


In [11]:
## Function 
def impute_nan(df,variable):
    mostfrequent_value = df[variable].value_counts().index[0]
    df[variable].fillna(mostfrequent_value,inplace=True)

In [12]:
for f in ['BsmtQual', 'FireplaceQu', 'GarageType']:
    impute_nan(df,f)

In [13]:
df.isnull().sum()

BsmtQual       0
FireplaceQu    0
GarageType     0
SalePrice      0
dtype: int64

## Advantages
    - easy to implement
## Disadvantages
    - replacing with the most frequent value for variables with many missing values may turn them into something over represented
    - distorts the relation of the most frequent value

## 2. Capture missing values with categorical variables

In [18]:
import numpy as np
import pandas as pd

In [22]:
df = pd.read_csv('house_prediction.csv',usecols=['BsmtQual', 'FireplaceQu', 'GarageType', 'SalePrice'])

In [23]:
df

,BsmtQual,FireplaceQu,GarageType,SalePrice
0,Gd,NaN,Attchd,208500
1,Gd,TA,Attchd,181500
2,Gd,TA,Attchd,223500
3,TA,Gd,Detchd,140000
4,Gd,TA,Attchd,250000
...,...,...,...,...
1455,Gd,TA,Attchd,175000
1456,Gd,TA,Attchd,210000
1457,TA,Gd,Attchd,266500
1458,TA,NaN,Attchd,142125


In [24]:
def impute_nan(df,variable):
    df[variable+'_new_var'] = np.where(df[variable].isnull(),'Missing', df[variable])

In [25]:
for f in ['BsmtQual', 'FireplaceQu', 'GarageType']:
    impute_nan(df,f)

In [26]:
## drop the original variables since we dont need them anymore
df.drop(['BsmtQual', 'FireplaceQu', 'GarageType'],axis=1,inplace=True)

In [27]:
df.head()

,SalePrice,BsmtQual_new_var,FireplaceQu_new_var,GarageType_new_var
0,208500,Gd,Missing,Attchd
1,181500,Gd,TA,Attchd
2,223500,Gd,TA,Attchd
3,140000,TA,Gd,Detchd
4,250000,Gd,TA,Attchd


## 3. Ordinal number encoding
- Suppose you have a grading system for a wine contest. The grades can be A, B or C, where A corresponds to the best wine. With this in mind, we can create ordinal variable like this:
    - A : 1
    - B: 2
    - C: 3

In [29]:
import datetime
import pandas as pd

In [36]:
today = datetime.datetime.today()

In [37]:
today - datetime.timedelta(1)

datetime.datetime(2021, 1, 8, 10, 10, 26, 217346)

In [39]:
##create 15 dates previous to today
days = [today - datetime.timedelta(x) for x in range(0,15)]

In [41]:
days_df = pd.DataFrame(days)

In [45]:
days_df.columns = ['day']

In [47]:
days_df.head()

,day
0,2021-01-09 10:10:26.217346
1,2021-01-08 10:10:26.217346
2,2021-01-07 10:10:26.217346
3,2021-01-06 10:10:26.217346
4,2021-01-05 10:10:26.217346


In [51]:
## how to retrieve weekday from a date
days_df.loc[0].dt.weekday_name

day    Saturday
Name: 0, dtype: object

In [53]:
days_df['weekday'] = days_df['day'].dt.weekday_name

In [55]:
days_df.head()

,day,weekday
0,2021-01-09 10:10:26.217346,Saturday
1,2021-01-08 10:10:26.217346,Friday
2,2021-01-07 10:10:26.217346,Thursday
3,2021-01-06 10:10:26.217346,Wednesday
4,2021-01-05 10:10:26.217346,Tuesday


In [56]:
## create dictionary to map the importance of each weekday
dict_weekday = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

In [59]:
days_df['weekday_order'] = days_df['weekday'].map(dict_weekday)

In [61]:
days_df.head()

,day,weekday,weekday_order
0,2021-01-09 10:10:26.217346,Saturday,6
1,2021-01-08 10:10:26.217346,Friday,5
2,2021-01-07 10:10:26.217346,Thursday,4
3,2021-01-06 10:10:26.217346,Wednesday,3
4,2021-01-05 10:10:26.217346,Tuesday,2


## 4. Frequency encoding

In [66]:
import pandas as pd
df = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',header=None)

In [67]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [91]:
df_cat = df.select_dtypes(include='object')

In [92]:
df_cat.columns = ['employment','degree','status','designation','parental_status','race','sex','country','annual_income']

In [93]:
df_cat

,employment,degree,status,designation,parental_status,race,sex,country,annual_income
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...
32556,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,United-States,<=50K
32557,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States,>50K
32558,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,United-States,<=50K
32559,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,United-States,<=50K


In [94]:
## chech how many categories there are in every feature
for col in df_cat.columns:
    print('Number of categories in {}: {}'.format(col, len(df_cat[col].unique())))

Number of categories in employment: 9
Number of categories in degree: 16
Number of categories in status: 7
Number of categories in designation: 15
Number of categories in parental_status: 6
Number of categories in race: 5
Number of categories in sex: 2
Number of categories in country: 42
Number of categories in annual_income: 2


In [97]:
## frequency enconding
dict_status = df_cat['status'].value_counts().to_dict()
df_cat = df_cat.copy()
df_cat['status'] = df_cat['status'].map(dict_status)

In [99]:
df_cat.head()

,employment,degree,status,designation,parental_status,race,sex,country,annual_income
0,State-gov,Bachelors,10683,Adm-clerical,Not-in-family,White,Male,United-States,<=50K
1,Self-emp-not-inc,Bachelors,14976,Exec-managerial,Husband,White,Male,United-States,<=50K
2,Private,HS-grad,4443,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
3,Private,11th,14976,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
4,Private,Bachelors,14976,Prof-specialty,Wife,Black,Female,Cuba,<=50K


In [100]:
dict_status

{' Married-civ-spouse': 14976,
 ' Never-married': 10683,
 ' Divorced': 4443,
 ' Separated': 1025,
 ' Widowed': 993,
 ' Married-spouse-absent': 418,
 ' Married-AF-spouse': 23}

## Advantages
    - Easy to implement
    - Not increasing the number of features
## Disadvantages
    - If 2 or more categories have the same frequency, they will be assigned to the same weight

## 4. Target guided ordinal encoding
- order the labels according to the target
- replace the labels by the joint probability of being 1 or 0

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('train.csv',usecols=['Survived','Cabin'])

In [5]:
df['Cabin'].fillna('Missing',inplace=True)

In [7]:
##retrieved only the block each cabin belongs to
df['Cabin'] = df['Cabin'].astype(str).str[0]

In [9]:
df.Cabin.unique()

array(['M', 'C', 'E', 'G', 'D', 'A', 'B', 'F', 'T'], dtype=object)

In [16]:
df.groupby('Cabin')['Survived'].mean().sort_values().index

Index(['T', 'M', 'A', 'G', 'C', 'F', 'B', 'E', 'D'], dtype='object', name='Cabin')

In [17]:
## the order of relevance of each cabin is based on the target (survived)
ordinal_labels = df.groupby('Cabin')['Survived'].mean().sort_values().index

In [18]:
ordinal_labels

Index(['T', 'M', 'A', 'G', 'C', 'F', 'B', 'E', 'D'], dtype='object', name='Cabin')

In [21]:
ordinal_labels2 = {k:i for i,k in enumerate(ordinal_labels,0)}

In [22]:
df['Cabin_ord_label'] = df['Cabin'].map(ordinal_labels2)

In [24]:
## we can now drop the Cabin column
df.drop('Cabin',axis=1,inplace=True)

In [25]:
df.head()

,Survived,Cabin_ord_label
0,0,1
1,1,4
2,1,1
3,1,4
4,0,1


## 5. Mean encoding

In [26]:
import pandas as pd

In [27]:
df = pd.read_csv('train.csv',usecols=['Survived','Cabin'])

In [28]:
df['Cabin'].fillna('Missing',inplace=True)

In [29]:
##retrieve only the block each cabin belongs to
df['Cabin'] = df['Cabin'].astype(str).str[0]

In [30]:
df.Cabin.unique()

array(['M', 'C', 'E', 'G', 'D', 'A', 'B', 'F', 'T'], dtype=object)

In [36]:
mean_ordinal_dict = df.groupby('Cabin')['Survived'].mean().to_dict()

In [37]:
df['mean_ordinal_encode'] = df['Cabin'].map(mean_ordinal_dict)

In [39]:
# we can now drop the Cabin column
df.drop('Cabin',axis=1, inplace=True)

## 6. Probability Ratio Encoding

In [40]:
import pandas as pd

In [41]:
df = pd.read_csv('train.csv',usecols=['Cabin', 'Survived'])

In [43]:
df['Cabin'].fillna('Missing',inplace=True)

In [46]:
df['Cabin'] = df['Cabin'].astype(str).str[0]

In [50]:
prob_df = pd.DataFrame(df.groupby('Cabin')['Survived'].mean())

In [51]:
## this dataframe shows the percentage of survivals based on the cabin block
prob_df

,Survived
Cabin,
A,0.466667
B,0.744681
C,0.593220
D,0.757576
E,0.750000
F,0.615385
G,0.500000
M,0.299854
T,0.000000


In [52]:
prob_df['Died'] = 1 - prob_df['Survived']

In [54]:
prob_df['Prob_ratio'] = prob_df['Survived']/prob_df['Died']

In [57]:
prob_ratio_dict = prob_df['Prob_ratio'].to_dict()

In [59]:
df['Cabin_prob_ratio'] = df['Cabin'].map(prob_ratio_dict)

In [61]:
df.drop('Cabin',axis=1,inplace=True)

In [62]:
df

,Survived,Cabin_prob_ratio
0,0,0.428274
1,1,1.458333
2,1,0.428274
3,1,1.458333
4,0,0.428274
...,...,...
886,0,0.428274
887,1,2.916667
888,0,0.428274
889,1,1.458333
